# How to use Whisper
usage is quite simple 2 step required:
1. open file in read "r" & binary "b" modes 
2. feed that file for whisper model 

In [14]:
from openai import OpenAI
from config import Config

# open file 
audio_file = open("various/task.mp3", "rb")

# call whisper model for that file
client = OpenAI(api_key=Config().open_api_key)
whisper_response = client.audio.transcriptions.create(
    model="whisper-1", 
    file=audio_file
)
transcript = whisper_response.text
print(f"Trancription => {transcript}")

Trancription => Cześć! Kiedy ostatnio korzystaliście z sztucznej inteligencji, czy zastanawialiście się nad tym, skąd czerpie ona swoją wiedzę? No pewnie, że tak, inaczej nie byłoby was tutaj na szkoleniu. Ale czy przemyśleliście możliwość dostosowania tej wiedzy do waszych własnych, indywidualnych potrzeb?


# Function Calling czyli generowania ustrukturyzowanych danych

documentation : https://platform.openai.com/docs/guides/function-calling

- jako dodatkowy parametr zapytania przekazujemy listę funkcji w postaci nazwy, opisu oraz zestaw parametrów
- poza tym przesyłamy także listę wiadomości, podobnie jak w klasycznej interakcji z modelem
- model w odpowiedzi zwraca nam nazwę funkcji oraz listę wartości jej parametrów
- opcjonalnie (ale jest to zwykle potrzebne) wskazujemy, która funkcja ma być wybrana jako domyślna

## Example
[make.com](https://medium.com/dev-bits/a-clear-guide-to-openai-function-calling-with-python-dcbc200c5d70#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6IjBlNzJkYTFkZjUwMWNhNmY3NTZiZjEwM2ZkN2M3MjAyOTQ3NzI1MDYiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIyMTYyOTYwMzU4MzQtazFrNnFlMDYwczJ0cDJhMmphbTRsamRjbXMwMHN0dGcuYXBwcy5nb29nbGV1c2VyY29udGVudC5jb20iLCJhdWQiOiIyMTYyOTYwMzU4MzQtazFrNnFlMDYwczJ0cDJhMmphbTRsamRjbXMwMHN0dGcuYXBwcy5nb29nbGV1c2VyY29udGVudC5jb20iLCJzdWIiOiIxMTQ4NzE3NjUwMjg0OTAzOTczMTkiLCJlbWFpbCI6InJhcGN6eW5za2lAZ21haWwuY29tIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsIm5iZiI6MTcwMDkyNjU4NywibmFtZSI6IkFydHVyIHJhcGN6ecWEc2tpIiwicGljdHVyZSI6Imh0dHBzOi8vbGgzLmdvb2dsZXVzZXJjb250ZW50LmNvbS9hL0FDZzhvY0pycmRKNzh1R0dyMmlMZ3Z5bVZma0tNU3hET3dDeThSQXlBUHpheDEtbz1zOTYtYyIsImdpdmVuX25hbWUiOiJBcnR1ciIsImZhbWlseV9uYW1lIjoicmFwY3p5xYRza2kiLCJsb2NhbGUiOiJlbiIsImlhdCI6MTcwMDkyNjg4NywiZXhwIjoxNzAwOTMwNDg3LCJqdGkiOiJjOTRkYTY1YWUwZWI4OTQ3M2ZlOWU3NmUwMjQ4MmFjOTJhNGM1YjBlIn0.UfqNI5JJcEwc48p8D189cMchHcoyy_inWyfzDHM0W1znomF_us55lmh7i3uEuYejtRFqmEuPXkqUQ_3OzlDxijJIOwIXbJOdvJWaAtoBWEpRqzmT0ipFfID3-WUUMDdiuwt-iTMHyvkkAyoDGTpJ28XDgY9LQkwUvjpcd_SfGB6JrlJDBJtYCXa3sfbyRyh5NLF69NJmdIHCJg1ztPlSWRGKKQvJL8lmBy3B1u4abm1XRwcH1eCcTtmOE_S6Ed3u2tThz-Tfb0LBMBJJszZk_MAPVNsofjfSQ2e93JlEflBmPhmmuoglNGO3OkeBjVL8iVWs3m4z3zX24raudl9KpQ)
first we create schema : 
PyDantic class to structure a model and convert it to JSON schema to avoid verbosity and errors.

goal of this excercise is to return structure answer to question: 
“Have a title and series of steps for a requested question, and generate a JSON-parsable output”.

{
  "properties": {
    "paragraph": {
      "title": "Paragraph",
      "type": "string"
    }
  },
  "required": [
    "paragraph"
  ],
  "title": "SimplePersonModel",
  "type": "object"
}


make an actual call to OpenAI and apply an “illusionary” function called “get_answer_for_user_query” by guiding AI to return JSON output
function is  `illusionary` because it’s control execution is automatically inferred by OpenAI model using name, description, and output schema but not defined by the developer.

In [68]:
from typing import List
from pydantic import BaseModel
import json

class SimplePersonModel(BaseModel):
    paragraph: str

simple_schema = SimplePersonModel.model_json_schema()
#print(json.dumps(simple_schema, indent=2))

client = OpenAI(api_key=Config().open_api_key)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "user", "content": "Write short paragraph about pirates"}
    ],
    functions=[
      {
          "name" : "provide_answer",
          "description" : "Get user answer",
          "parameters" : SimplePersonModel.model_json_schema()
      }
    ],
    function_call={"name": "provide_answer"}
)

#respone for list 
print(response.choices[0].message)
output = json.loads(response.choices[0].message.function_call.arguments)
print(json.dumps(output, indent=2))


{
  "properties": {
    "paragraph": {
      "title": "Paragraph",
      "type": "string"
    }
  },
  "required": [
    "paragraph"
  ],
  "title": "SimplePersonModel",
  "type": "object"
}
ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "paragraph": "Pirates were notorious figures who sailed the seas, seeking adventure and plundering ships. These seafaring outlaws were known for their distinctive clothing, eye patches, and peg legs. They would often fly the Jolly Roger, a skull and crossbones flag, to strike fear into the hearts of their victims. Pirates were skilled at navigation and combat, using their swords and pistols to attack unsuspecting vessels. They would seize valuable cargo such as gold, silver, and precious jewels, making them incredibly wealthy. Despite their ruthless reputation, pirates also lived by their own code of honor and had a strong sense of camaraderie. While piracy was eventually suppressed, the legends and lor

 ## this is more complex example with list 

In [69]:
from typing import List
from pydantic import BaseModel
import json

class StepByStepAIResponse(BaseModel):
    title: str
    steps: List[str]

list_schema = StepByStepAIResponse.model_json_schema() 

# print(json.dumps(list_schema, indent=2))

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "Act as senior joke specialis"},
      {"role": "user", "content": "Give receipt for most hilarious and triggering joke ever"}
    ],
    functions=[
        {
          "name": "get_answer_for_user_query",
          "description": "Get user answer in series of steps",
          "parameters": list_schema 
        }
    ],
    function_call={"name": "get_answer_for_user_query"}
)

output = json.loads(response.choices[0].message.function_call.arguments)
print(json.dumps(output, indent=2))

{
  "properties": {
    "title": {
      "title": "Title",
      "type": "string"
    },
    "steps": {
      "items": {
        "type": "string"
      },
      "title": "Steps",
      "type": "array"
    }
  },
  "required": [
    "title",
    "steps"
  ],
  "title": "StepByStepAIResponse",
  "type": "object"
}
{
  "title": "Receipt for Hilarious and Triggering Joke",
  "steps": [
    "Step 1: Enter the setup of the joke",
    "Step 2: Enter the punchline of the joke",
    "Step 3: Enter the trigger warning for the joke (if applicable)"
  ]
}


## Clasification example from aidevs

In [78]:
from typing import List
import pydantic 
import json
import enum

class TypeEnum(enum.Enum):
    MEMO = "memory"
    NOTE = "notes"
    LINK = "links"
    TODO = "todo"
  

class ClasificationModel(BaseModel):
    command: bool 
    type: TypeEnum = pydantic.Field(...)
    tags : List[str]

list_schema = ClasificationModel.model_json_schema() 

# print(json.dumps(list_schema, indent=2))


def call_ai(): 
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {"role": "user", "content": "write report for monday "}
        ],
        functions=[
            {
            "name": "clasify_answer",
            "description": "Describe users query with semantic tags and classify with type",
            "parameters": list_schema 
            }
        ],
        function_call={"name": "clasify_answer"}
    )

    output = json.loads(response.choices[0].message.function_call.arguments)
    print(json.dumps(output, indent=2))

call_ai()

{
  "command": true,
  "type": "todo",
  "tags": [
    "report"
  ]
}
